
# Exercice 1 : le terminal et l'arborescence

<details>
    <summary>** Voir la solution de la question 1 **</summary>

`$ cd ~` ou `$ cd` ou `$ cd /home/login`

puis ```$ ls -lrt```
</details>
<details>
<summary>** Voir la solution de la question 2 **</summary>

`$ mkdir arborescence`
</details>

<details>
    <summary>** Voir la solution de la question 3 **</summary>
    
```
$ cp Downloads/arborescence.tar arborescence/
$ cd arborescence
$ tar -xvf arborescence.tar
```
</details>
<details>
    <summary>** Voir la solution de la question 4 **</summary>

```$ find -iname "CP7fQ5.txt"```

```$ cat ./sdVoQL/70LqBL/CP7fQ5.txt``` la réponse est donc : 42 !
</details>
 <details>
    <summary>** Voir la solution de la question 5 **</summary>

```$ du -h``` je trouve 440 Ko

```$ df -h``` le 12 août 2023, on trouvait :
    
```
Filesystem                               Size  Used Avail Use% Mounted on
home.cbp.ens-lyon.fr:/home               4.0T  2.6T  1.5T  63% /home
scratch.cbp.ens-lyon.fr:/scratch          79T   42T   38T  53% /scratch
```
</details>

# Exercice 2 : droits sur les fichiers
<details>
    <summary>** Voir la solution de la question 1 **</summary>
    
```$ cd```

puis ```$ cd ..``` et ```ls```
</details>
<details>
    <summary>** Voir la solution de la question 2 **</summary>

```$ cd```

```$ mkdir python``` puis ```$ mkdir terminal``` ou de manière plus condensée ```$ mkdir -p python/terminal```
</details>

## Exercice 3 : créer un alias et l'activer
<details>
    <summary>** Voir la solution de la question 1 **</summary>
    
```ls -a```
</details>

<details>
    <summary>** Voir la solution de la question 2 **</summary>
    
```gedit ~/.bashrc &```
</details>

<details>
    <summary>** Voir la solution de la question 3 **</summary>

Ajouter les lignes suivantes :   
```
#mes alias
alias ll='ls -lrt'
```
    
Puis ```source ~/.bashrc```
</details>

## Exercice 4 : se connecter à distance via ssh
<details>
    <summary>** Voir la solution de la question 1 **</summary>
    
```ssh mverot@ssh.ens-lyon.fr``` avec votre login à la place de `mverot`
</details>
<details>
    <summary>** Voir la solution de la question 2 **</summary>
    
```scp mverot@k20m.cbp.ens-lyon.fr:~/python/terminal/default.py .``` avec votre login à la place de `mverot` et n'importe quelle machine listée ici : https://www.cbp.ens-lyon.fr/python/forms/CloudCBP à la place de `k20m`.
    
```rm default.py``` pour le supprimer.
    
```exit``` pour fermer la connexion ssh.
</details>



## Exercice 5 : un peu de scripting en bash
<details>
    <summary>** Voir la solution de la question 1 **</summary>
Le plus simple est encore ... d'aller voir la documentation https://legacy.imagemagick.org/Usage/resize/#shrink
    
    
```
#!/bin/bash

for f in *.jpg
do
    echo "processing $f ..."
    convert $f -resize 100x100\> ${f%.jpg}-thumbnail.jpg
done

mkdir thumbnails
mv *-thumbnail.jpg thumbnails    
```
</details>
<details>
    <summary>** Voir la solution de la question 2 **</summary>


    
```
#!/bin/bash

for f in *.png
do
echo "processing $f ..."

convert $f  -white-threshold 85% -black-threshold 40% ${f%.png}_clean.png
convert ${f%.png}_clean.png ${f%.png}_clean.pdf
done

pdfunite *_clean.pdf scans.pdf
```
</details>


<details>
    <summary>** Voir la solution de la question 3 **</summary>


    
```
#!/bin/bash

for f in *.png
do
echo "processing $f ..."

#convert $f  -white-threshold 85% -black-threshold 40% ${f%.png}_clean.png
tesseract "${f%.png}_clean.png" "${f%.png}_1" -l fra
cat ${f%.png}_1.txt >> scan.txt
done
```
</details>

## Exercice 6 : Pour aller plus loin : notion de PATH, automatisation
<details>
    <summary>** Voir la solution de la question 1 **</summary>
    
```
$ cd 
$ mkdir moldy
$ mv Downloads/exoO1-moldy-2.16e.tgz moldy
$ cd moldy
$ tar -xvf moldy-2.16e.tgz
$ gedit READ.ME &
$ ./configure
$ make
```
</details>
<details>
    <summary>** Voir la solution de la question 2 **</summary>
   
```
$ export PATH="$PATH:$HOME/moldy"   
ou depuis le dossier $ export PATH="$PATH:."  
$ moldy control.argon
$ moldy control.argon > output.dat
```
</details>
<details>
    <summary>** Voir la solution de la question 3**</summary>

```
$ cd
$ mkdir moldy-simulations
$ cp moldy/control.argon moldy-simulations
$ gedit launch-moldy.sh &
```

```
#!/bin/bash

for temp in $(seq 84 5 134)
do
    echo "Température : $temp"
    sed -e "s/temperature=84/temperature=$temp/g" control.argon > control.argon.$temp
    moldy control.argon.$temp > "output-$temp.out"
done
```
</details>

<details>
    <summary>** Voir la solution de la question 4**</summary>

### Avec `grep` 

 * -P permet d'utiliser des expressions régulières, 
 * -o permet d'afficher uniquement les lignes correspondant au motif, 
 * `\K` permet de n'afficher que la partie qui correspond à la suite. 

```
#!/bin/bash
echo "Temperature,Pressure" >> TvsP.csv
for temp in $(seq 84 5 134)
do
    echo "Température : $temp"
    Pressure=$(grep -Po 'Pressure\s*=\s*\K[^+]*' output-$temp*)
    Temperature=$(grep -Po '^\s*Temp\s*=\s*\K[^+\s]*' output-$temp*)
    echo $Temperature,$Pressure >> TvsP.csv
done
```

### Avec `sed` et `cut` 
 * p en fin de commande permet d'afficher les lignes correspondantes
 * on utilise le pipe pour enchainer le traitement (enlever l'incertitude)
 * -d indique le délimiteur pour cut
 * -f indique le numéro de champ que l'on veut récupérer 
 
```
#!/bin/bash
echo "Temperature,Pressure" >> TvsP.csv
for temp in $(seq 84 5 134)
do
    echo "Température : $temp"
    Pressure=$(sed -n  's/Pressure *= *//p' output-$temp* | cut -d '+' -f 1)
    Temperature=$(sed -n 's/^ *Temp *= *//p' output-$temp* | cut -d '+' -f 1)
    echo $Temperature,$Pressure >> TvsP.csv
done
```

### Avec `sed` uniquement 
On utilise des expressions régulières : 
 * `^` indique le début de ligne (pour la température pour éviter les autres températures)
 * `\(\)` permet de faire des parenthèses capturantes
 * `\1` est le résultat de la capture

```
#!/bin/bash
echo "Temperature,Pressure" >> TvsP.csv
for temp in $(seq 84 5 134)
do
    echo "Température : $temp"
    Pressure=$(sed -n  's/Pressure *= *\([^ ]*\) *+.*/\1/p' output-$temp* )
    Temperature=$(sed -n 's/^ *Temp *= *\([^ ]*\) *+.*/\1/p' output-$temp* )
    echo $Temperature,$Pressure >> TvsP.csv
done
```

### Avec `grep` et `awk`
`awk` permet de sélectionner le troisième mot (entité séparée par des espaces) de la ligne extraite par `grep`
Ici, les espaces sont importants pour ne sélectionner que certaines lignes.


```
#!/bin/bash
echo "Temperature,Pressure" >> TvsP.csv
for temp in $(seq 84 5 134)
do
    echo "Température : $temp"
    Pressure=$(grep "Pressure  " output-$temp* | awk '{print $3}')
    Temperature=$(grep ' Temp  ' output-$temp* | awk '{print $3}')
    echo $Temperature,$Pressure >> TvsP.csv
done
```


</details>